In [1]:
import pandas as pd

In [2]:
fips_url_full = 'https://www.dropbox.com/s/04eu0q6sgph8wto/US_FIPS_Codes.xls?dl=1'
fips_url_abv = 'https://www.dropbox.com/s/d503mesxlsc1yfa/county_fips.csv?dl=1'
fl_pop_path = '../20_intermediate_files/fl_shipment_pop.csv'
fl_path = '../20_intermediate_files/florida_shipment_cleaned.csv'

In [90]:
fl_df = pd.read_csv(fl_path)
fl_df.shape

(1720, 4)

In [4]:
fips_full = pd.read_excel(fips_url_full, header=1, dtype={'State':str, 'County Name':str,'FIPS State':str, 'FIPS County':str})
fips_full['FIPS Code'] = fips_full['FIPS State'] + fips_full['FIPS County']
fips_full['FIPS Code'] = fips_full['FIPS Code'].str.lstrip('0')
fips_full.drop(columns=['FIPS State', 'FIPS County'], inplace=True)
fips_full['FIPS Code'] = fips_full['FIPS Code'].astype(int)
fips_full.head()

,State,County Name,FIPS Code
0,Alabama,Autauga,1001
1,Alabama,Baldwin,1003
2,Alabama,Barbour,1005
3,Alabama,Bibb,1007
4,Alabama,Blount,1009


In [5]:
fips_abv = pd.read_csv(fips_url_abv)
fips_abv.rename(columns={'countyfips': 'FIPS Code'}, inplace=True)
fips_abv

,BUYER_COUNTY,BUYER_STATE,FIPS Code
0,AUTAUGA,AL,1001
1,BALDWIN,AL,1003
2,BARBOUR,AL,1005
3,BIBB,AL,1007
4,BLOUNT,AL,1009
...,...,...,...
3138,WASHAKIE,WY,56043
3139,WESTON,WY,56045
3140,SKAGWAY,AK,2230
3141,HOONAH ANGOON,AK,2105


In [6]:
fips = fips_abv.merge(fips_full, how = 'left', on=['FIPS Code'], validate="1:1")

In [7]:

fips.drop(columns=['County Name'],inplace=True)
#NaN for AK
fips.isna().sum()

BUYER_COUNTY    0
BUYER_STATE     0
FIPS Code       0
State           3
dtype: int64

In [8]:
missing_state = fips['State'].isna()
missing_state

0       False
1       False
2       False
3       False
4       False
        ...  
3138    False
3139    False
3140     True
3141     True
3142     True
Name: State, Length: 3143, dtype: bool

In [9]:
impute_dict=dict({'AK':'Arkansas'})
fips.loc[missing_state, 'State'] = fips.loc[missing_state, 'BUYER_STATE'].map(impute_dict)
print(fips.isna().sum())
fips.shape

BUYER_COUNTY    0
BUYER_STATE     0
FIPS Code       0
State           0
dtype: int64


(3143, 4)

In [10]:
fl_ship_fips = fl_df.merge(fips, how='left', on=['BUYER_STATE','BUYER_COUNTY'], validate="m:1")
fl_ship_fips

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State
0,AZ,APACHE,2006,4.956969e+06,4001,Arizona
1,AZ,APACHE,2007,5.611684e+06,4001,Arizona
2,AZ,APACHE,2008,5.853244e+06,4001,Arizona
3,AZ,APACHE,2009,7.344332e+06,4001,Arizona
4,AZ,APACHE,2010,7.732743e+06,4001,Arizona
...,...,...,...,...,...,...
1715,SC,YORK,2010,5.209517e+07,45091,South Carolina
1716,SC,YORK,2011,5.296504e+07,45091,South Carolina
1717,SC,YORK,2012,5.644417e+07,45091,South Carolina
1718,SC,YORK,2013,5.813704e+07,45091,South Carolina


In [67]:
fl_df_pop = pd.read_csv(fl_pop_path)
fl_df_pop['STNAME'].unique()


array(['Arizona', 'Florida', 'Louisiana', 'South Carolina'], dtype=object)

In [68]:
fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.replace(' County', '')
fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.upper()
fl_df_pop.CTYNAME.unique()

array(['APACHE', 'COCHISE', 'COCONINO', 'GILA', 'GRAHAM', 'GREENLEE',
       'LA PAZ', 'MARICOPA', 'MOHAVE', 'NAVAJO', 'PIMA', 'PINAL',
       'SANTA CRUZ', 'YAVAPAI', 'YUMA', 'ALACHUA', 'BAKER', 'BAY',
       'BRADFORD', 'BREVARD', 'BROWARD', 'CALHOUN', 'CHARLOTTE', 'CITRUS',
       'CLAY', 'COLLIER', 'COLUMBIA', 'DESOTO', 'DIXIE', 'DUVAL',
       'ESCAMBIA', 'FLAGLER', 'FRANKLIN', 'GADSDEN', 'GILCHRIST',
       'GLADES', 'GULF', 'HAMILTON', 'HARDEE', 'HENDRY', 'HERNANDO',
       'HIGHLANDS', 'HILLSBOROUGH', 'HOLMES', 'INDIAN RIVER', 'JACKSON',
       'JEFFERSON', 'LAFAYETTE', 'LAKE', 'LEE', 'LEON', 'LEVY', 'LIBERTY',
       'MADISON', 'MANATEE', 'MARION', 'MARTIN', 'MIAMI-DADE', 'MONROE',
       'NASSAU', 'OKALOOSA', 'OKEECHOBEE', 'ORANGE', 'OSCEOLA',
       'PALM BEACH', 'PASCO', 'PINELLAS', 'POLK', 'PUTNAM', 'ST. JOHNS',
       'ST. LUCIE', 'SANTA ROSA', 'SARASOTA', 'SEMINOLE', 'SUMTER',
       'SUWANNEE', 'TAYLOR', 'UNION', 'VOLUSIA', 'WAKULLA', 'WALTON',
       'WASHINGTON', 'ACA

In [78]:
fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.replace(' PARISH', '')
fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.replace('ST. ', 'SAINT ')
fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.replace('DESOTO', 'DE SOTO')
fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.replace('LASALLE', 'LA SALLE')
fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.replace('SAINT JOHN THE BAPTIST', 'ST JOHN THE BAPTIST')
fl_df_pop['CTYNAME'].unique()

/var/folders/n2/5x35hhp53jg1kc9jxswsqhww0000gn/T/ipykernel_62355/1753056515.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  fl_df_pop['CTYNAME'] = fl_df_pop['CTYNAME'].str.replace('ST. ', 'SAINT ')


array(['APACHE', 'COCHISE', 'COCONINO', 'GILA', 'GRAHAM', 'GREENLEE',
       'LA PAZ', 'MARICOPA', 'MOHAVE', 'NAVAJO', 'PIMA', 'PINAL',
       'SANTA CRUZ', 'YAVAPAI', 'YUMA', 'ALACHUA', 'BAKER', 'BAY',
       'BRADFORD', 'BREVARD', 'BROWARD', 'CALHOUN', 'CHARLOTTE', 'CITRUS',
       'CLAY', 'COLLIER', 'COLUMBIA', 'DE SOTO', 'DIXIE', 'DUVAL',
       'ESCAMBIA', 'FLAGLER', 'FRANKLIN', 'GADSDEN', 'GILCHRIST',
       'GLADES', 'GULF', 'HAMILTON', 'HARDEE', 'HENDRY', 'HERNANDO',
       'HIGHLANDS', 'HILLSBOROUGH', 'HOLMES', 'INDIAN RIVER', 'JACKSON',
       'JEFFERSON', 'LAFAYETTE', 'LAKE', 'LEE', 'LEON', 'LEVY', 'LIBERTY',
       'MADISON', 'MANATEE', 'MARION', 'MARTIN', 'MIAMI-DADE', 'MONROE',
       'NASSAU', 'OKALOOSA', 'OKEECHOBEE', 'ORANGE', 'OSCEOLA',
       'PALM BEACH', 'PASCO', 'PINELLAS', 'POLK', 'PUTNAM', 'SAINT JOHNS',
       'SAINT LUCIE', 'SANTA ROSA', 'SARASOTA', 'SEMINOLE', 'SUMTER',
       'SUWANNEE', 'TAYLOR', 'UNION', 'VOLUSIA', 'WAKULLA', 'WALTON',
       'WASHINGTON',

In [81]:
fl_df_pop

,STNAME,CTYNAME,YEAR,POPULATION
0,Arizona,APACHE,2000,69516
1,Arizona,COCHISE,2000,118028
2,Arizona,COCONINO,2000,116717
3,Arizona,GILA,2000,51355
4,Arizona,GRAHAM,2000,33541
...,...,...,...,...
3835,South Carolina,SPARTANBURG,2019,320254
3836,South Carolina,SUMTER,2019,106635
3837,South Carolina,UNION,2019,27221
3838,South Carolina,WILLIAMSBURG,2019,30256


In [84]:
fl_pop_fips = fl_df_pop.merge(fips, how='left', left_on=['STNAME', 'CTYNAME'], right_on=['State', 'BUYER_COUNTY'])
fl_pop_fips.isna().sum()

STNAME          0
CTYNAME         0
YEAR            0
POPULATION      0
BUYER_COUNTY    0
BUYER_STATE     0
FIPS Code       0
State           0
dtype: int64

In [88]:
yo = fl_pop_fips.merge(fl_ship_fips, how='right', on=['FIPS Code', 'YEAR'])
yo.isna().sum()

STNAME            0
CTYNAME           0
YEAR              0
POPULATION        0
BUYER_COUNTY_x    0
BUYER_STATE_x     0
FIPS Code         0
State_x           0
BUYER_STATE_y     0
BUYER_COUNTY_y    0
MME               0
State_y           0
dtype: int64

# CHECKING WASHINGTON

In [16]:
wa_path = '../20_intermediate_files/washington_shipment_cleaned.csv'
wa_pop_path = '../20_intermediate_files/wa_shipment_pop.csv'

In [17]:
wa_df = pd.read_csv(wa_path)
wa_ship_fips = wa_df.merge(fips, how="left", on=["BUYER_STATE", "BUYER_COUNTY"], validate="m:1")
wa_ship_fips

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State
0,AZ,APACHE,2006,4.956969e+06,4001,Arizona
1,AZ,APACHE,2007,5.611684e+06,4001,Arizona
2,AZ,APACHE,2008,5.853244e+06,4001,Arizona
3,AZ,APACHE,2009,7.344332e+06,4001,Arizona
4,AZ,APACHE,2010,7.732743e+06,4001,Arizona
...,...,...,...,...,...,...
1565,WA,YAKIMA,2010,6.746887e+07,53077,Washington
1566,WA,YAKIMA,2011,7.600735e+07,53077,Washington
1567,WA,YAKIMA,2012,8.023265e+07,53077,Washington
1568,WA,YAKIMA,2013,8.391631e+07,53077,Washington


In [18]:

#clean and standardize population data to merge with shipment
wa_df_pop = pd.read_csv(wa_pop_path)
wa_df_pop['CTYNAME'] = wa_df_pop['CTYNAME'].str.replace(' County', '')
wa_df_pop['CTYNAME'] = wa_df_pop['CTYNAME'].str.upper()
wa_df_pop

,STNAME,CTYNAME,YEAR,POPULATION
0,Arizona,APACHE,2000,69516
1,Arizona,COCHISE,2000,118028
2,Arizona,COCONINO,2000,116717
3,Arizona,GILA,2000,51355
4,Arizona,GRAHAM,2000,33541
...,...,...,...,...
3595,Washington,WAHKIAKUM,2019,4420
3596,Washington,WALLA WALLA,2019,61062
3597,Washington,WHATCOM,2019,228675
3598,Washington,WHITMAN,2019,50136


In [19]:

#merge shipment and population data
wa_ship_pop = wa_ship_fips.merge(wa_df_pop, how='left', left_on=['State', 'BUYER_COUNTY','YEAR'], right_on=['STNAME', 'CTYNAME', 'YEAR'], validate="1:1", copy=False)

In [20]:
wa_ship = pd.read_csv('../20_intermediate_files/wa_ship_merge.csv')
wa_ship.isna().sum()

BUYER_STATE     0
BUYER_COUNTY    0
YEAR            0
MME             0
FIPS Code       0
State           0
STNAME          9
CTYNAME         9
POPULATION      9
dtype: int64

In [21]:
fl_ship = pd.read_csv('../20_intermediate_files/fl_ship_merge.csv')
fl_ship.isna().sum()

BUYER_STATE       0
BUYER_COUNTY      0
YEAR              0
MME               0
FIPS Code         0
State             0
STNAME          598
CTYNAME         598
POPULATION      598
dtype: int64

In [22]:
fl_ship[fl_ship.STNAME.isna()]['BUYER_COUNTY'].unique()

array(['DE SOTO', 'SAINT JOHNS', 'SAINT LUCIE', 'ACADIA', 'ALLEN',
       'ASCENSION', 'ASSUMPTION', 'AVOYELLES', 'BEAUREGARD', 'BIENVILLE',
       'BOSSIER', 'CADDO', 'CALCASIEU', 'CALDWELL', 'CAMERON',
       'CATAHOULA', 'CLAIBORNE', 'CONCORDIA', 'EAST BATON ROUGE',
       'EAST CARROLL', 'EAST FELICIANA', 'EVANGELINE', 'FRANKLIN',
       'GRANT', 'IBERIA', 'IBERVILLE', 'JACKSON', 'JEFFERSON',
       'JEFFERSON DAVIS', 'LA SALLE', 'LAFAYETTE', 'LAFOURCHE', 'LINCOLN',
       'LIVINGSTON', 'MADISON', 'MOREHOUSE', 'NATCHITOCHES', 'ORLEANS',
       'OUACHITA', 'PLAQUEMINES', 'POINTE COUPEE', 'RAPIDES', 'RED RIVER',
       'RICHLAND', 'SABINE', 'SAINT BERNARD', 'SAINT CHARLES',
       'SAINT HELENA', 'SAINT JAMES', 'SAINT LANDRY', 'SAINT MARTIN',
       'SAINT MARY', 'SAINT TAMMANY', 'ST JOHN THE BAPTIST', 'TANGIPAHOA',
       'TENSAS', 'TERREBONNE', 'UNION', 'VERMILION', 'VERNON',
       'WASHINGTON', 'WEBSTER', 'WEST BATON ROUGE', 'WEST CARROLL',
       'WEST FELICIANA', 'WINN'], dtype

In [23]:
wa_ship[wa_ship.STNAME.isna()]
#population does not have SAINT LAWRENCE, 

,BUYER_STATE,BUYER_COUNTY,YEAR,MME,FIPS Code,State,STNAME,CTYNAME,POPULATION
1057,NY,SAINT LAWRENCE,2006,2.186481e+07,36089,New York,NaN,NaN,NaN
1058,NY,SAINT LAWRENCE,2007,2.305936e+07,36089,New York,NaN,NaN,NaN
1059,NY,SAINT LAWRENCE,2008,2.414880e+07,36089,New York,NaN,NaN,NaN
1060,NY,SAINT LAWRENCE,2009,2.740978e+07,36089,New York,NaN,NaN,NaN
1061,NY,SAINT LAWRENCE,2010,2.894361e+07,36089,New York,NaN,NaN,NaN
1062,NY,SAINT LAWRENCE,2011,3.018991e+07,36089,New York,NaN,NaN,NaN
1063,NY,SAINT LAWRENCE,2012,2.992549e+07,36089,New York,NaN,NaN,NaN
1064,NY,SAINT LAWRENCE,2013,3.069736e+07,36089,New York,NaN,NaN,NaN
1065,NY,SAINT LAWRENCE,2014,3.062440e+07,36089,New York,NaN,NaN,NaN
